In [ ]:
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy

In [ ]:
# To avoid OOM errors, setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(f"GPU: {gpu}")
    tf.config.experimental.set_memory_growth(gpu, True) # Keeping the use of memory limited to prevent errors

In [ ]:
data_dir = '../data/processed'
img_ex = cv2.imread(os.path.join(data_dir, 'a', 'a_FATIMA_9_front.JPG'))
img_ex.shape

In [ ]:
plt.imshow(cv2.cvtColor(img_ex, cv2.COLOR_BGR2RGB))

In [ ]:
# Automatically creates a dataset form the referred directory. Load the full dataset, shuffle = True ensures randomness
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(256, 256),
    batch_size=32,
    shuffle=True,
    seed=123
)

# In order to iterare the element we must use the iterative method
data_iterator = dataset.as_numpy_iterator()
data_iterator

In [ ]:
batch = data_iterator.next() # Grabbing a dataset batch

# Exploring a batch labels
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

### Data visualization